# Neuromorpho.org 2 blender notebook

This Notebook accompanies the neuromorpho2blender_headless.py script

It let's you preview neurons of interest in 2d and 3d, and control the running of a headless blender instance to create and render a blend file.

# Imports

In [1]:
import numpy as np
import pandas as pd
import pickle
import subprocess as sp # For running blender in headless mode

from neuromorpholib import neuromorpho

# For showing the png of the neuron in the notebook
from IPython.display import Image
from IPython.core.display import HTML 

# For visualizing the neuron in 3d
import plotly.express as px
import plotly.graph_objects as go

# Functions

In [ ]:
def load_data(n_neurons=121544): 

    filename = 'first_'+str(n_neurons)+'_neurons_'
    df = pd.read_pickle(filename+'df.pkl') 

    with open(filename+'measurments_list.pkl', 'rb') as f:
        measurements = pickle.load(f)
        
    with open(filename+'info_listdf.pkl', 'rb') as f:
        info = pickle.load(f)    

    return df, measurements, info

In [ ]:
def show_neuron_png(neur_id):    
    
    this_row = df.loc[df['neuron_id'] == neur_id]

    neur_ind = this_row.index[0] # Distinct from neur_id, but one value off. This is the value used to index the lists collected previously.
                                # zero-indexed to remove dtype of pandas index    # print(neur_ind, np.shape(neur_ind))

    # Based on the id, get the neuron name, archive, and image url
    this_name = info[neur_ind]['neuron_name']
    this_archive = info[neur_ind]['archive']
    this_url = info[neur_ind]['png_url']
    print(this_url)
    
    # View the .png version with html
    img = Image(url= this_url)
    display(img)


In [ ]:
def show_neuron_3d(neur_id, export=False):    
    this_row = df.loc[df['neuron_id'] == neur_id]

    neur_ind = this_row.index[0] # Distinct from neur_id, but one value off. This is the value used to index the lists collected previously.
                                # zero-indexed to remove dtype of pandas index

    # Based on the id, get the neuron name, archive, and image url
    this_name = info[neur_ind]['neuron_name']
    this_archive = info[neur_ind]['archive']
    this_url = info[neur_ind]['png_url']

    # Use neuromorpho to get the associated swc file
    nmo = neuromorpho.NeuroMorpho()
    test_swc = nmo.download_swc(this_archive, this_name, text_only=True)

    # Save the swc locally as a temporary file (Otherwise save as a named file)
    text_file = open("tmp.swc", "wt")
    n = text_file.write(test_swc)
    text_file.close()

    # Load the swc file as numpy arrays
    neur_data = np.loadtxt('tmp.swc')
    x = neur_data[:,2]
    y = neur_data[:,3]
    z = neur_data[:,4]

    # Plot with Plotly
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                       mode='markers',
                                       marker=dict(
                                       size=2,
                                       color=z,                # set color to an array/list of desired values
                                       colorscale='Viridis',   # choose a colorscale
                                       opacity=0.8
                                       ))])
    fig.show()
    if(export):
        fig.write_html('neuron_'+str(neur_id)+'.html')

In [ ]:
def blender_animation(neur_id):
    
    blender_path = 'C://Program Files/Blender Foundation/Blender/blender.exe'
    blend_file = 'neuron_rotation_template.blend'
    script_name = 'neuromorpho2blender_headless.py'
    sp.call([blender_path,'--background',blend_file,'--python',script_name, '--', str(neur_id)])


In [ ]:
df, measurements, info = load_data() 
display(df)

In [ ]:
neur_id = 42574
show_neuron_png(neur_id)

In [ ]:
show_neuron_3d(neur_id)

In [ ]:
blender_animation(neur_id)